# Twitter Thesis Project: collecting tweets 

## Introduction
This is a 'jupyter notebook': a certain kind of program you can use to develop your own software applications. In this notebook we will use the Python computer language and the Twitter API ('application programmer interface') to automatically collect and analyse tweets. 

This notebook contains cells, i.e., snippets of either code or normal text. We use the text cells (like the current one) to explain what is going on. You can edit a cell by clicking on it. After you made the changes, you can either click 'run' above, or press shift+enter, to execute what is written. In the case of a text cell, this will just display the text in the correct format (try it with this cell!); in case of a code cell, the code will be executed. 

The next cell will be a code cell where we ask the computer to print a simple sentence for us. Try to change this sentence and then execute the code. 


In [2]:
print('hello world')

hello world


If you want to learn more about using jupyter notebooks, look for a tutorial online (e.g., https://www.dataquest.io/blog/jupyter-notebook-tutorial/). Most of the questions you have or the problems you encounter will also be solved through a simple google search with the correct keywords.

**However, if you have other questions or any problems that you really don't know how to solve, please send an email to hpinson@vub.be and I'll be happy to help or schedule a meeting. **

## Connecting to the Twitter API

In [1]:
from twython import Twython
#if this results in an error, you need to install twython first. 

First, we need to connect to Twitter using the correct passwords/keys. There is a limit on how many tweets you can collect each 15 minutes (this makes sure the Twitter servers are not overloaded, amongst other reasons).  Running the code below 'logs you in' to the Twitter application. If all goes well, the output should show information on the number of calls ('questions we can ask') we can still perform these 15 minutes. With each call, you can collect 100 Tweets. 
e.g.: {'/search/tweets': {'limit': 450, 'remaining': 443, 'reset': 1568288620}}

In [2]:
APP_KEY = 'yN3VbAb8QZdzD5GPkVuOHLfMN'         #API key
APP_SECRET = 'YRdyk39bx9iRPQBhK2Nh1fT32JdGYTrEhqxcEbcpLMIxbT7wKh'   #API secret key
twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()

twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)
twitter.get_application_rate_limit_status()['resources']['search']



{'/search/tweets': {'limit': 450, 'remaining': 450, 'reset': 1568894597}}

## Searching Tweets

In [26]:
#result = twitter.search(q='python', result_type='popular')
#result = twitter.search(q='vrije universiteit brussel', count='100')

#zoek naar 100 (het maximum) tweets die het woord 'brussel' bevatten; uitgezonderd retweets. 
resultBrussels = twitter.search(q='brussel -filter:retweets', count='100')

resultAntwerpen = twitter.search(q='antwerpen -filter:retweets', count='100')

In [27]:
for x in resultBrussels['statuses']:
  print(x['text'])
  break


Tester Cucumber - Schaarbeek, Brussel https://t.co/NDW2f6tyZf


In [28]:
for x in resultAntwerpen['statuses']:
    print('--')
    print(x['text'])

--
Super een besluit wat de @PubliekeOmroep veel eerder had moeten nemen.
In België is de nationale intocht in Antwerp… https://t.co/VG4173Waye
--
Zgadza się: Antwerpen Centraal ma cztery poziomy, z których pociągi są na trzech https://t.co/IZLvVfjIUR
--
A2see #Hoogeveen heeft een LED screen geplaatst in de etalage van Van Haren te Antwerpen. Dit LED screen wordt inge… https://t.co/Ae6DbjNkvr
--
@KatoCassiers 25,5 miljoen op 2 jaar heeft de Stad Antwerpen in hun kas geslagen aan boetes. Echt schandalig. En to… https://t.co/nGav1MsVOS
--
Educatief Medewerker Kazou Provincie Antwerpen – Mechelen https://t.co/pUjA8b6Gzn
--
This place is absolutely in the middle of nowhere. (@ Mol in Antwerpen) https://t.co/igGJScc2ug
--
Studeer je IT, design, marketing en/of sales en zoek je een uitdagende &amp; leerrijke stageplaats in een innovatief be… https://t.co/G3dWf6Cksr
--
See @GreenDay  with @8bitMicroFan (Best Brother ever) in Antwerpen. I can't wait 😃😃😃 #HellaMegaTour https://t.co/jhp096UAFc
-

## Streaming Tweets

In [3]:
from twython import TwythonStreamer
import csv
import os.path


filename = 'collected_tweets_keyword_brussel.csv'




In [4]:
class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        print("-------new tweet collected!")
        
        if 'retweeted_status' in data:
            print("but it's a retweet...")
        else:
      
            print([data['created_at'],data['place'], data['text']])
            
            file_exists = os.path.isfile(filename)
         
            with open(filename,'a') as f: #this will add the newly collected tweets to your dataset ('a' = append) 
                writer = csv.writer(f)
                if not file_exists: 
                    writer.writerow(['Date','Place','Text'])
                writer.writerow([data['created_at'],data['place'], data['text']])
            

    def on_error(self, status_code, data):
        #print(data)
        print(status_code)
        # self.disconnect()

In [ ]:
OAUTH_TOKEN = '1100028871259377670-qtcMTW2ereJ3A0KIvFguWu0ZmW0n8k'
OAUTH_TOKEN_SECRET = 'wnPYmWOds9xD1i1CM9K8gfzMNZ26QoBmXW4JSSA81faRF'


stream = MyStreamer(APP_KEY, APP_SECRET,
                    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
stream.statuses.filter(track='brussel')

-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 11:49:29 +0000 2019', {'id': '6cc4ad9ab1459208', 'url': 'https://api.twitter.com/1.1/geo/id/6cc4ad9ab1459208.json', 'place_type': 'city', 'name': 'Taradell', 'full_name': 'Taradell, España', 'country_code': 'ES', 'country': 'España', 'bounding_box': {'type': 'Polygon', 'coordinates': [[[2.241318, 41.849725], [2.241318, 41.904215], [2.331924, 41.904215], [2.331924, 41.849725]]]}, 'attributes': {}}, 'Salgamos de la UE. \nYA.']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 11:50:58 +0000 2019', None, 'I’m ready for us to be in a serous relationship 🔥😍🤣']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 11:53:37 +0000 2019', None, 'Zo, weer even snel een boel treintickets gekocht voor de komende maand voor Brussel en Parijs enzo. Lekker elektris… https://t.co/i3BA9V

-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 12:28:49 +0000 2019', None, 'GwH herstelt de taalwetgeving in ere. Minister had in de wet moeten inschrijven dat de rechter moet kunnen ingrijpe… https://t.co/FHhAOSeimS']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 12:31:04 +0000 2019', None, 'Brussel - Kathmandu van 603 euro H/T;  https://t.co/1amMHZMQqA https://t.co/gHuzzYjK4n']
-------new tweet collected!
['Thu Sep 19 12:31:34 +0000 2019', None, 'Was hankering for a filet last night. Topped it with some Camembert and sauteed mixed mushrooms. Those are purple b… https://t.co/WihlCS7AdZ']
-------new twee

-------new tweet collected!
['Thu Sep 19 13:08:29 +0000 2019', None, 'Timing Is Everything\nNew blog post at https://t.co/L9RIyfAuxu on how this steak dinner with baby brussel sprouts an… https://t.co/TliH6V6jJc']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 13:09:06 +0000 2019', None, '@petervanderhey6 @PhRoose Peter with all due respect, géén enkele ploeg in géén enkele sport komt graag naar Antwer… https://t.co/Hve5mBNIW1']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 13:10:46 +0000 2019', None, "@murmi38 @boye_g Ja es pot esgargamellar el Boye ja que si l'ase no vol veure! El Club de Brussel·les es.... això!"]
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's

-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 13:52:31 +0000 2019', None, 'Ensor in Brussel: maskers, schelpen en stoute prenten\xa0\xa0 https://t.co/AlcarPzCWm #vrtnws']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 13:53:20 +0000 2019', None, 'i want brussel sprouts.']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 13:54:51 +0000 2019', None, '@repgregwalden @am860TheAnswer Representative Walden, way to take on the real, tough issues facing our country. Can… https://t.co/9vZSO1xPvd']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 13:57:10 +0000 2019', None, 'Ensor in Brussel: maskers, schelpen en stoute prenten\xa0\xa0 https://t.co/1jI2FstUfr https://t.co/jC7LNW

-------new tweet collected!
['Thu Sep 19 14:25:29 +0000 2019', None, 'So many faves in one pic 😍😍😍😫😫😫']
-------new tweet collected!
['Thu Sep 19 14:25:41 +0000 2019', None, 'Een plezierige eerste vergadering gehad met de collega gedeputeerden EU. Uitstekend bijgepraat op lopende dossiers… https://t.co/AUTs4blBCt']
-------new tweet collected!
['Thu Sep 19 14:25:49 +0000 2019', None, 'Brexitpapirer levert i Brussel – rettssak avsluttet i\xa0London https://t.co/txLvcV88cs https://t.co/UJxo1tnN7d']
-------new tweet collected!
['Thu Sep 19 14:26:05 +0000 2019', None, 'Eveline Crone over The promise of a new generation bij viering van het 12,5 jarig bestaan van Neth-er in Brussel https://t.co/7fQHRwxEDP']
-------new tweet collected!
['Thu Sep 19 14:26:13 +0000 2019', None, '@vincemindfreak @HomerElStoner @TeenagePie DEEP FRIED BRUSSEL SPROUTS \n👀\n👅']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 14:27:36 +0000 2019', None, '#onderwijs in #brussel

-------new tweet collected!
['Thu Sep 19 14:54:06 +0000 2019', None, 'Garlic Bread']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 14:56:52 +0000 2019', None, '#E40 Ongeval in Aalter richting Brussel. De rechterrijstrook is versperd.']
-------new tweet collected!
['Thu Sep 19 14:58:52 +0000 2019', None, 'Lexus Lounge Brussel beste van Europa https://t.co/gm1DOujW9x']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 14:59:29 +0000 2019', {'id': '9f1ffdda79b5299e', 'url': 'https://api.twitter.com/1.1/geo/id/9f1ffdda79b5299e.json', 'place_type': 'city', 'name': 'Ans', 'full_name': 'Ans, Belgique', 'country_code': 'BE', 'country': 'Belgium', 'bounding_box': {'type': 'Polygon', 'coordinates': [[[5.460671, 50.648331], [5.460671, 50.722748], [5.552482, 50.722748], [5.552482, 50.648331]]]}, 'attributes': {}}, '#E40 (A3) Luik richting Brussel , ter hoogte van 

-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 15:27:02 +0000 2019', None, 'Elke week op een ander plein in Brussel.']
-------new tweet collected!
['Thu Sep 19 15:28:19 +0000 2019', None, '@RetiredDA @NATO “European Flavor” ahem Brussel’s (power) Flavor, cough cough (up)  billions $$$$$...']
-------new tweet collected!
['Thu Sep 19 15:28:39 +0000 2019', None, 'Drukke avondspits rond Brussel. Bij Hoeilaart is bovendien een ongeluk gebeurd op de ringweg #R0. \n#Belgie https://t.co/iHGl60z0VX']
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 15:29:24 +0000 2019', None, "@daisyschaut What's the difference between bogies and Brussel Sprouts?\n\nYou can get a kid to eat bogies"]
-------new tweet collected!
but

-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 15:56:33 +0000 2019', {'id': '276a8de8953329e6', 'url': 'https://api.twitter.com/1.1/geo/id/276a8de8953329e6.json', 'place_type': 'city', 'name': 'Aa en Hunze', 'full_name': 'Aa en Hunze, Nederland', 'country_code': 'NL', 'country': 'Nederland', 'bounding_box': {'type': 'Polygon', 'coordinates': [[[6.561452, 52.868154], [6.561452, 53.094752], [6.936366, 53.094752], [6.936366, 52.868154]]]}, 'attributes': {}}, '@Toon_Sesink @BNR Het zijn onze eigen stikstof normen. Zelf bedacht en daarna in gediend bij Brussel.  Daarna houdt… https://t.co/Za0fe3rvNH']
-------new tweet collected!
['Thu Sep 19 15:56:42 +0000 2019', None, "gonna go with a classic: brussel sprouts. when they're cooked right, they're delicious. \nafter living on my own, i'… https://t.co/TCg4bHd76I"]
-------new tweet collected!
but it's a retweet...
-------new tweet collected!
['Thu Sep 19 15:58:22 +0000 2019', None, '@Magali_DeReu Dat 

-------new tweet collected!
['Thu Sep 19 16:29:09 +0000 2019', {'id': '75431660bc651ca9', 'url': 'https://api.twitter.com/1.1/geo/id/75431660bc651ca9.json', 'place_type': 'city', 'name': 'Breda', 'full_name': 'Breda, Nederland', 'country_code': 'NL', 'country': 'The Netherlands', 'bounding_box': {'type': 'Polygon', 'coordinates': [[[4.658036, 51.476718], [4.658036, 51.643137], [4.871663, 51.643137], [4.871663, 51.476718]]]}, 'attributes': {}}, '#Beneluxtrein 9256 (V 18:42 van Breda) naar Brussel-Zuid is 6 minuten vertraagd. #BNL9256']
